In [1]:
import pandas as pd
 
# read text file into pandas DataFrame
X_trainpd = pd.read_csv("D:/Mini project/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt", delim_whitespace=True, header=None).values
y_trainpd = pd.read_csv("D:/Mini project/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt", delim_whitespace=True, header=None).values.ravel()
X_testpd = pd.read_csv("D:/Mini project/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt", delim_whitespace=True, header=None).values
y_testpd = pd.read_csv("D:/Mini project/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt", delim_whitespace=True, header=None).values.ravel()

In [2]:
X_combined = pd.concat([pd.DataFrame(X_trainpd), pd.DataFrame(X_testpd)])
y_combined = pd.concat([pd.Series(y_trainpd), pd.Series(y_testpd)])

In [31]:
type(X_combined)

pandas.core.frame.DataFrame

In [3]:
import numpy as np
import pandas as pd
from deap import base, creator, tools, algorithms

num_features = X_combined.shape[1]
def attribute_from_dataset():
    return X_combined
    
# Fuzzy Logic Rules
def gaussian_membership(x, x_hat, theta):
    return np.exp(-((x - x_hat) ** 2) / (2 * theta ** 2))

def fuzzy_rules(features):
    average, maximum, minimum, std_deviation, variance = features
    # Fuzzy rules
    fuzzy_classification = None
    if all(avg > -0.6 and mini > 0.2 and maxi > -0.7 and std > 0.1 for avg, mini, maxi, var, std in zip(average, minimum, maximum, variance, std_deviation)):
        fuzzy_classification = "BEST"
    elif all(avg > -0.6 and mini > 0.2 and maxi > -0.7 and std <=0.1 for avg, mini, maxi, var, std in zip(average, minimum, maximum, variance, std_deviation)):
        fuzzy_classification = "GOOD"
    else:
        fuzzy_classification = "BAD"

    return fuzzy_classification

# Genetic Algorithm Setup
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_float",attribute_from_dataset)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=num_features)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# Define fuzzy fitness function
# Define fuzzy fitness function
   


def fuzzy_fitness(individual):
    # Extract features
    selected_columns = [i for i in range(561)]
    feature=np.take(X_combined,selected_columns,axis=1)
    average = np.mean(feature,axis=0)
    maximum = np.max(feature,axis=0)
    minimum = np.min(feature,axis=0)
    std_deviation = np.std(feature,axis=0)
    variance = np.var(feature,axis=0)
    features=[average,maximum,minimum,std_deviation,variance]
    # Apply fuzzy rules
    fuzzy_classification = fuzzy_rules(features)
    gaussian_values = []
    for col_avg, col_std in zip(average, std_deviation):
        gaussian_value = gaussian_membership(col_avg, 0.5, col_std) 
        gaussian_values.append(gaussian_value)

    # Calculate weighted sum of Gaussian values as fitness
    gaussian_value=np.prod(gaussian_values)
    # Assign fitness values based on the fuzzy classification
    if fuzzy_classification == 'BEST':
        fitness_value = 0.4 *gaussian_value
    elif fuzzy_classification == 'GOOD':
        fitness_value = 0.3 *gaussian_value
    else:
        fitness_value = 0.3

    return (fitness_value,)


toolbox.register("evaluate", fuzzy_fitness)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.2, indpb=0.2)

# Tournament Selection Function
def tournament_selection(population, k):
    tournsize = 5  # Specify the value of tournsize here
    selected = []

    # Extract indices of individuals in the population
    indices = list(range(len(population)))

    for _ in range(k):
        competitors_indices = np.random.choice(indices, tournsize, replace=False)
        competitors = [population[i] for i in competitors_indices]
        selected.append(max(competitors, key=lambda ind: ind.fitness.values[0]))
    
    return selected

toolbox.register("select", tournament_selection)
hof = tools.HallOfFame(1)
# Genetic Algorithm

population_size = X_combined.shape[0]  # Set population size equal to the number of rows in your dataset
population = toolbox.population(n=population_size)  # Initialize population with rows of the dataset
population,log=algorithms.eaMuPlusLambda(population, toolbox, mu=population_size, lambda_=2*population_size, cxpb=0.7, mutpb=0.2, ngen=0, stats=None, halloffame=hof)

best_individuals = hof




gen	nevals
0  	10299 


MemoryError: Unable to allocate 44.1 MiB for an array with shape (561, 10299) and data type float64

In [68]:
population[0]

[           0         1         2         3         4         5         6    \
 0     0.288585 -0.020294 -0.132905 -0.995279 -0.983111 -0.913526 -0.995112   
 1     0.278419 -0.016411 -0.123520 -0.998245 -0.975300 -0.960322 -0.998807   
 2     0.279653 -0.019467 -0.113462 -0.995380 -0.967187 -0.978944 -0.996520   
 3     0.279174 -0.026201 -0.123283 -0.996091 -0.983403 -0.990675 -0.997099   
 4     0.276629 -0.016570 -0.115362 -0.998139 -0.980817 -0.990482 -0.998321   
 ...        ...       ...       ...       ...       ...       ...       ...   
 2942  0.310155 -0.053391 -0.099109 -0.287866 -0.140589 -0.215088 -0.356083   
 2943  0.363385 -0.039214 -0.105915 -0.305388  0.028148 -0.196373 -0.373540   
 2944  0.349966  0.030077 -0.115788 -0.329638 -0.042143 -0.250181 -0.388017   
 2945  0.237594  0.018467 -0.096499 -0.323114 -0.229775 -0.207574 -0.392380   
 2946  0.153627 -0.018437 -0.137018 -0.330046 -0.195253 -0.164339 -0.430974   
 
            7         8         9    ...       551

In [69]:
import pandas as pd
df1=pd.DataFrame(population[0])

In [76]:
df2=pd.DataFrame(population[0][0])

In [79]:
df2.head(7356)

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7351,0.351503,-0.012423,-0.203867,-0.269270,-0.087212,0.177404,-0.377404,-0.038678,0.229430,0.269013,...,-0.058402,-0.387052,-0.740738,-0.280088,-0.007739,-0.056088,-0.616956,-0.783267,0.246809,0.036695
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,0.071645,-0.330370,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.401189,-0.121845,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,0.062891,-0.190422,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346


In [80]:
df2.to_csv('geneticalgooutput.csv')